<a href="https://colab.research.google.com/github/Param-Bhatt/Bert-Search/blob/main/Bert.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
%cd "/content/drive/My Drive/IR Assignment 1"

/content/drive/My Drive/IR Assignment 1


In [4]:
!pip install datasets

     |████████████████████████████████| 153kB 11.6MB/s 
     |████████████████████████████████| 245kB 26.4MB/s 
     |████████████████████████████████| 17.7MB 197kB/s 
  Found existing installation: pyarrow 0.14.1
    Uninstalling pyarrow-0.14.1:
      Successfully uninstalled pyarrow-0.14.1


In [5]:
!pip install bert-embedding --no-deps

In [6]:
!pip install gluonnlp

     |████████████████████████████████| 348kB 10.9MB/s 
  Created wheel for gluonnlp: filename=gluonnlp-0.10.0-cp36-cp36m-linux_x86_64.whl size=588514 sha256=fda191b62daad44f2feadd43844f950a767d2271b00abcb25a029416f2aa32cd
  Stored in directory: /root/.cache/pip/wheels/37/65/52/63032864a0f31a08b9a88569f803b5bafac8abd207fd7f7534
Successfully built gluonnlp
ERROR: bert-embedding 1.0.1 requires mxnet==1.4.0, which is not installed.
ERROR: bert-embedding 1.0.1 requires typing==3.6.6, which is not installed.
ERROR: bert-embedding 1.0.1 has requirement gluonnlp==0.6.0, but you'll have gluonnlp 0.10.0 which is incompatible.
ERROR: bert-embedding 1.0.1 has requirement numpy==1.14.6, but you'll have numpy 1.18.5 which is incompatible.


In [7]:
!pip install mxnet-cu101

     |███████████████████████████████▌| 834.1MB 1.3MB/s eta 0:00:09tcmalloc: large alloc 1147494400 bytes == 0x396fe000 @  0x7f4f86632615 0x591f47 0x4cc229 0x4cc38b 0x50a51c 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50cfd6 0x58e793 0x50c467 0x58e793 0x50c467 0x58e793 0x50c467 0x58e793 0x50c467 0x509918 0x50a64d 0x50c1f4 0x507f24 0x509c50 0x50a64d 0x50c1f4 0x509918 0x50a64d
     |████████████████████████████████| 846.0MB 22kB/s 
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


In [8]:
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer 
nltk.download('wordnet')
nltk.download('stopwords')
lemmatizer = WordNetLemmatizer()
import re
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;!@]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))
def preprocess(text):
    """
    string: a string
    return: preprocessed string
    Takes text as input and 
        1) Makes everything lower case.
        2) Deletes '[/(){}\[\]\|@,;!@]' and then '[^0-9a-z #+_]'.
        3) Removes StopWords.
        4) Lemmatizes every word.
    """
    text = text.lower()
    text = REPLACE_BY_SPACE_RE.sub(" " , text)  # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub("", text)         # delete symbols which are in BAD_SYMBOLS_RE from text
    textlist = text.split()
    text = [wrd for wrd in textlist if wrd not in STOPWORDS]
    text = " ".join(text)
    
    #Lemmatize
    lem_words = []
    for word in text.split():
        lem_words.append(lemmatizer.lemmatize(word))
    return " ".join(lem_words)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [9]:
#Actual Dataset
import numpy as np
from datasets import load_dataset

dataset = load_dataset("aeslc")

Using custom data configuration default


Dataset aeslc downloaded and prepared to /root/.cache/huggingface/datasets/aeslc/default/1.0.0/0f335d7ea7f9ab9e325673a86c6aa0e2cd503c5dbf17d48fddcb02840a92fc96. Subsequent calls will reuse this data.


In [10]:
s = dataset['train'][:]['email_body']
len(s)

14436

In [11]:
#Emails as 
arr = []
for i in range(len(s)):
    arr.append([i for i in s[i].split('\n') if len(i)!=0 ])
assert 14436 == len(arr)
arr=None

In [12]:
corpus = []
for email in s:
    email = email.split("\n")
    #print(email)
    p=[]
    #print()
    for i in range(len(email)):
        if len(email[i]) > 0:
            prep_s = preprocess(email[i])
            if len(prep_s) > 0:
                p.append(prep_s)
    corpus.append(p)
len(corpus)

14436

In [13]:
import mxnet as mx
from bert_embedding import BertEmbedding

ctx = mx.gpu(0)
be = BertEmbedding(ctx=ctx)

Vocab file is not found. Downloading.


In [14]:
embeddings = []
for email in range(len(corpus)):
    printc=True
    for i in be(corpus[email]):
        if email%500==0 and printc:
            print("Iteration: ", email)
            printc=False
        #print(i[1])
        #print(sum(i[1]))
        #print(len(i[1]))
        try:
            embeddings.append(sum(i[1])/len(i[1]))
        except:
            print(i, email, corpus[email])

Iteration:  0
Iteration:  500
Iteration:  1000
Iteration:  1500
Iteration:  2000
Iteration:  2500
Iteration:  3000
Iteration:  3500
Iteration:  4000
Iteration:  4500
Iteration:  5000
Iteration:  5500
Iteration:  6000
Iteration:  6500
Iteration:  7000
Iteration:  7500
Iteration:  8000
Iteration:  8500
Iteration:  9000
Iteration:  9500
Iteration:  10000
Iteration:  10500
Iteration:  11000
Iteration:  11500
Iteration:  12000
Iteration:  12500
Iteration:  13000
Iteration:  13500
Iteration:  14000


In [15]:
#Save Data to PKL file
import pickle

with open('Bert_Email_Embeddings.pkl', 'wb') as fp:
    pickle.dump(embeddings, fp) 
len(embeddings)

105406

In [16]:
with open ('Bert_Email_Embeddings.pkl', 'rb') as fp:
    itemlist = pickle.load(fp)
len(itemlist)

105406

In [17]:
len(corpus)

14436

In [18]:
l=0 #INDEX THROUGH EVERY SENTENCE
email_emb=[] #VEC OF EMAIL
eml = []
for email in corpus:
    sarr = [] #SENTENCE VECS
    for _ in email:
        if len(preprocess(_)) > 0:
            eml.append(_)
            sarr.append(embeddings[l])
            l+=1
    email_emb.append(sum(sarr)/len(sarr))
len(sarr), l

(4, 105337)

In [19]:
indx_to_email = dict()
l=0
for indx_email in range(len(corpus)):
    start = l
    for j in range(len(corpus[indx_email])):
        indx_to_email[l]=(start,indx_email)
        l+=1
len(indx_to_email), l

(105406, 105406)

In [20]:
def find_email(i_email, highlight_line=None):
    for i in range(len(s[i_email])):
        if i==highlight_line:
            print('\33[34m' + s[i_email][i] + '\033[0m')
        else:
            print(s[i_email][i])

def cosine_score(vec1, vec2):
    return np.dot(vec1, vec2)/(np.linalg.norm(vec1) * np.linalg.norm(vec2))

def search(query, top_n=3):    
    l = 0
    for q in query:
        c = [i for i in q.split('\n') if len(i)!=0]
        que = [preprocess(i) for i in q.split('\n') if len(i)!=0 ]
        emb = be(que)
        for i in range(len(emb)):
            print("\33[35mQuery {} of {} \033[0m".format(l+1, len(emb)))
            e = emb[i][1]
            embb = sum(e)/len(e)
            similarity = []
            for i in itemlist:
                similarity.append(cosine_score(embb, i))
            len(similarity)
            sim = np.argsort(similarity)[::-1][:top_n]
            print("Query:", "\33[31m" + c[l] + "\033[0m", "\n")
            l+=1
            for i in sim:
                print("\33[33m" + "Cosine Similarity: {:.3f} \t Email {} \t Line {} \033[0m".format(similarity[i], indx_to_email[i][1], i-indx_to_email[i][0]))
                print("\33[34m" + corpus[indx_to_email[i][1]][i-indx_to_email[i][0]] + '\033[0m',  "\n")

In [21]:
query = ["I will fill in the Legal description of the property one I have received it. \n Please let me know how you wish to handle this and I will proceed."]
search(query, top_n=4)


Query 1 of 2 
Query: I will fill in the Legal description of the property one I have received it.  

Cosine Similarity: 1.000 	 Email 0 	 Line 2 
fill legal description property one received 

Cosine Similarity: 0.822 	 Email 12610 	 Line 0 
jeff received request counterparty alter format confirmation process physical financial deal 

Cosine Similarity: 0.816 	 Email 906 	 Line 13 
would appear work backwards summary sale unsecured creditor committee want 

Cosine Similarity: 0.815 	 Email 4450 	 Line 1 
one letter lieu transfer division order linder dunhill regarding certain property sold dunhill resource llc 11 01 01 

Query 2 of 2 
Query:  Please let me know how you wish to handle this and I will proceed. 

Cosine Similarity: 1.000 	 Email 1 	 Line 2 
please let know wish handle proceed 

Cosine Similarity: 0.899 	 Email 11645 	 Line 1 
please let know want handle default 

Cosine Similarity: 0.880 	 Email 9792 	 Line 3 
please let know handle issue 

Cosine Similarity: 0.875 	 Emai

In [22]:
search(["Just thought you might want to put it on the list of fun  things to do in the future."])

Query 1 of 1 
Query: Just thought you might want to put it on the list of fun  things to do in the future. 

Cosine Similarity: 1.000 	 Email 7512 	 Line 1 
thought might want put list fun thing future 

Cosine Similarity: 0.898 	 Email 7055 	 Line 1 
point well thought would like day get new draft 

Cosine Similarity: 0.890 	 Email 141 	 Line 8 
let know friend think might want play 



In [23]:
search(["I have made some really good friends and I would like to stay in touch with all of you."])

Query 1 of 1 
Query: I have made some really good friends and I would like to stay in touch with all of you. 

Cosine Similarity: 1.000 	 Email 9999 	 Line 3 
made really good friend would like stay touch 

Cosine Similarity: 0.858 	 Email 7512 	 Line 1 
thought might want put list fun thing future 

Cosine Similarity: 0.855 	 Email 7055 	 Line 1 
point well thought would like day get new draft 

